<a href="https://colab.research.google.com/github/laurenttang/colab/blob/main/Train_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Dec 28 11:07:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#!pip install keras==2.4.3
#!pip install tensorflow==2.4.0
!pip install keras==2.2.4
!pip install tensorflow==1.13.1

In [3]:
import keras
import tensorflow 
print(keras.__version__)
print(tensorflow.__version__)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

2.2.4
1.13.1


In [4]:
#!/usr/bin/env python
# coding: utf-8


# -*- coding: utf-8 -*-
"""
Created on Wed May 15 14:13:24 2019

@author: user
"""

from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,ZeroPadding2D
from keras.layers import Activation, Dense




model = Sequential() #建立模型主體

model.add(Conv2D(                        #第一層 捲積層1 輸入層
                 filters = 32,           #建立出32個濾鏡
                 kernel_size = (3,3),    #每個濾鏡3X3大小
                 activation ='relu',     #定義 激活函數 為'relu'
                 padding = 'same',       #此捲積層產生捲積影像大小不變
                 input_shape = (29,60,3) #輸入層的資料入口格式 高29 寬60 RGB 3
                )
         )

model.add(MaxPool2D(pool_size=(2,2)))    #第二層 池化層1 縮減取樣

model.add(Dropout(0.2))                 #避免過度擬合 捨去20%的神經元

model.add(Conv2D(                        #第三層 捲積層2
                 filters = 64,           #建立出64個濾鏡
                 kernel_size = (3,3),    #每個濾鏡3X3大小
                 activation ='relu',     #定義 激活函數 為'relu'
                 padding = 'same',       #此捲積層產生捲積影像大小不變
                )
         )

model.add(MaxPool2D(pool_size=(2,2)))    #第四層 池化層2 縮減取樣

model.add(Dropout(0.2))                 #避免過度擬合 捨去20%的神經元

model.add(Flatten())                     #平坦層將以上特徵(3個維)轉換為1維

model.add(Dropout(0.2))                 #避免過度擬合 捨去20%的神   經元

model.add( Dense(                       #加入 神經網路層1 隱藏層
                 units =10,             #此 神經網路層2 隱藏層 的神經元個數 
                 activation ='relu'     #定義 激活函數 為'relu'
                )
         )

model.add(Dropout(0.2))                 #避免過度擬合 捨去20%的神經元

model.add(Dense(                        #加入 神經網路層2 輸出層
                 units =5,              #此 神經網路層2 輸出層 依照答案有5個
                 activation ='softmax'  #定義 激活函數 為'softmax'
                )
         )

print(model.summary()) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 29, 60, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 30, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 30, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 15, 64)         0         
_________________________________________________________________
dropout_2 (Dropout)  

In [5]:
## use colab
#from google.colab import drive
#import os
#drive.mount('/content/drive')

##出現提示欄進行授權

#os.chdir('/content/drive/My Drive/Colab') #切換該目錄
#os.listdir('test/') #確認目錄內容
## ==

In [6]:
import numpy as np
import pandas as pd
import cv2
import os
import h5py

## use colab
from google.colab import drive
import os
drive.mount('/content/drive')

##出現提示欄進行授權

os.chdir('/content/drive/My Drive/Colab') #切換該目錄
#os.listdir('test/') #確認目錄內容
## ==

ar = list(range(1, 169))              #我們創建一個1~40的空間

x_image_train=[]                     #裝圖片的變數
y_label_train = []                   #裝答案的變數

Long = os.listdir("TrainSample/Resizedata/Long")
Short = os.listdir("TrainSample/Resizedata/Short")
Swing = os.listdir("TrainSample/Resizedata/Swing")
WBottom = os.listdir("TrainSample/Resizedata/WBottom")

for index in Long :                    #跑回圈 ar =1~40
    img =  cv2.imread("TrainSample/Resizedata/Long/"+index)          #使用CV2讀取圖片
    x_image_train.append(img)        #將圖片加入裝圖片的變數
    y_label_train.append([1])
for index in Short :                    #跑回圈 ar =1~40
    img =  cv2.imread("TrainSample/Resizedata/Short/"+index)          #使用CV2讀取圖片
    x_image_train.append(img)        #將圖片加入裝圖片的變數
    y_label_train.append([2])
for index in Swing :                    #跑回圈 ar =1~40
    img =  cv2.imread("TrainSample/Resizedata/Swing/"+index)          #使用CV2讀取圖片
    x_image_train.append(img)        #將圖片加入裝圖片的變數
    y_label_train.append([3])                                  
for index in WBottom :                    #跑回圈 ar =1~40
    img =  cv2.imread("TrainSample/Resizedata/WBottom/"+index)          #使用CV2讀取圖片
    x_image_train.append(img)        #將圖片加入裝圖片的變數
    y_label_train.append([4])
        
x_image_train=np.asarray(x_image_train)    #轉成array
y_label_train =np.asarray(y_label_train)
#for i in range(0,len(x_image_train)):
#    print(type(x_image_train[i]))
print(x_image_train.shape)                #查看arrayru結構，40張圖片+相片高29像素+相片寬60像素+RGB 3原色

x_image_train = x_image_train.astype('float32') / 255.0  #將數字影像標準化

print(x_image_train)

from keras.utils import np_utils
y_label_train = np_utils.to_categorical(y_label_train) #答案以Onehot encoding 轉換
print(y_label_train)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(672, 29, 60, 3)
[[[[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         0.99607843 1.        ]
   [1.         0.99607843 1.        ]
   [0.9764706  0.972549   0.98039216]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [1.         0.99607843 1.        ]
   [1.         0.99607843 1.        ]
   [1.         0.99607843 1.        ]]

  [[1.         1.         1.        ]
   [1.         1.         1.        ]
   [1.         1.         1.        ]
   ...
   [0.9372549  0.93333334 0.9411765 ]
   [0.9764706  0.972549   0.98039216]
   [1.         0.99607843 1.        ]]

  ...

  [[0.9764706  0.99215686 0.972549  ]
   [0.9764706  0.99215686 0.972549  ]
   [0.5294118  0.54509807 0.5254902 ]
   ...
   [1.         

In [7]:
##定義模型訓練方式
from keras import metrics
model.compile(                                 #設定模型
              loss='mean_squared_error',       #損失函數 此例使用 均方誤差
              optimizer = 'adam',              #優化方法 此例使用 adam
              metrics =['accuracy']            #評估模型的方式 此例我們希望的是準確率()
             )
model.optimizer.lr.assign(0.05)

train_model = model.fit(x = x_image_train,             #資料裡對應的特徵
                        y = y_label_train,             #資料裡對應的結果
                        epochs = 50000,                   #訓練週期 此例設定為20
                        batch_size = 512,                #訓練批次筆數 此例設定為2
                        verbose = 2                    #顯示訓練過程
                       )



#保存模型
model.save('Recognize_Ver4.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/50000
 - 3s - loss: 0.1617 - acc: 0.2202
Epoch 2/50000
 - 2s - loss: 0.1557 - acc: 0.2634
Epoch 3/50000
 - 2s - loss: 0.1555 - acc: 0.2649
Epoch 4/50000
 - 2s - loss: 0.1551 - acc: 0.2470
Epoch 5/50000
 - 2s - loss: 0.1541 - acc: 0.2619
Epoch 6/50000
 - 2s - loss: 0.1544 - acc: 0.2411
Epoch 7/50000
 - 2s - loss: 0.1529 - acc: 0.2783
Epoch 8/50000
 - 2s - loss: 0.1518 - acc: 0.2946
Epoch 9/50000
 - 2s - loss: 0.1504 - acc: 0.3571
Epoch 10/50000
 - 2s - loss: 0.1516 - acc: 0.3006
Epoch 11/50000
 - 2s - loss: 0.1481 - acc: 0.3735
Epoch 12/50000
 - 2s - loss: 0.1481 - acc: 0.3810
Epoch 13/50000
 - 2s - loss: 0.1452 - acc: 0.4494
Epoch 14/50000
 - 2s - loss: 0.1419 - acc: 0.4167
Epoch 15/50000
 - 2s - loss: 0.1374 - acc: 0.5030
Epoch 16/50000
 - 2s - loss: 0.1339 - acc: 0.5298
Epoch 17/50000
 - 2s - loss: 0.1302 - acc: 0.5164
Epoch 18/50000
 - 2s - loss: 0.1254 - acc: 0.5595
Epoch 19/50000
 - 2s - loss: 0.1196 - acc: 0.6027
Epoch 20/50

KeyboardInterrupt: ignored

In [ ]:
from google.colab import files

files.download('Recognize_Ver4.h5')